# Simple Word Tokenizers

In [62]:
import re

In [63]:
# Reading and examining the toy dataset
with open('../DataPreparation/the-verdict.txt','r',encoding='utf-8') as f:
    raw_text = f.read()
print("Total number of characters: ",len(raw_text))
print(raw_text[0:99])

Total number of characters:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


## Generating Vocabulary from Dataset

In [64]:
# Simple Tokenizer using Regular Expressions
# Splits on whitespaces, commas, periods, question marks, quotation marks, 
# double-dashes, colons, semi-colons and other special characters.
# This is followed by removal of extra whitespaces
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4690


In [65]:
# Creating a vocabulary

allwords = sorted(set(preprocessed))
vocab_size = len(allwords)
vocab = {token:integer for integer,token in enumerate(allwords)}

## Simple Tokenizer with Original Vocabulary

In [66]:
class SimpleTokenizerV1:
    #1 Stores the vocabulary as a class attribute for access in the encode and decode methods
    #2 Creates an inverse vocabulary that maps token IDs back to the original text tokens
    #3 Processes input text into token IDs
    #4 Converts token IDs back into text
    #5 Removes spaces before the specified punctuation
    def __init__(self, vocab):
        self.str_to_int = vocab #1
        self.int_to_str = {i: s for s, i in vocab.items()} #2

    def encode(self, text): #3
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids): #4
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r"\1", text) #5
        return text

In [67]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know," 
       Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [68]:
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


## Augmenting Vocabulary to account for Unknown Tokens and Represent end of Document

In [69]:
# Creating a vocabulary

all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}

print(len(vocab.items()))

1132


## Simple Tokenizer with Augmented Vocabulary

In [70]:
class SimpleTokenizerV2:
    # 1 Replaces unknown words by <|unk|> tokens
    # 2 Replaces spaces before the specified punctuations
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int else "<|unk|>" for item in preprocessed  # 1
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])

        text = re.sub(r'\s+([,.:;?!"()\'])', r"\1", text)  # 2
        return text

In [71]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [72]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [73]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


# Byte Pair Encoding

In [74]:
import tiktoken

In [75]:
tokenizer = tiktoken.get_encoding("gpt2")
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145
